In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import os
import cv2
import keras
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [ ]:
import keras
from tensorflow.keras.utils import img_to_array,array_to_img,  load_img
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
food_labels=os.listdir('/content/drive/MyDrive/machine learning/Food classification/food11/train')
food_labels

In [ ]:

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
AugFolderPath='/content/drive/MyDrive/machine learning/Food classification/food11/ALL_AUG_images'
Train_Images_path='/content/drive/MyDrive/machine learning/Food classification/food11/train'
for i in food_labels:
  folder_path=os.path.join(Train_Images_path,i)
  for j in os.listdir(folder_path):
    img = load_img(os.path.join(folder_path,j))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    k = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=os.path.join(AugFolderPath,i), save_prefix=i, save_format='jpeg'):
        k += 1
        if k > 1:
          break

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
AugFolderPath='/content/drive/MyDrive/machine learning/Food classification/food11/ALL_AUG_images'
Test_Images_path='/content/drive/MyDrive/machine learning/Food classification/food11/test'
for i in food_labels:
  folder_path=os.path.join(Test_Images_path,i)
  for j in os.listdir(folder_path):
    img = load_img(os.path.join(folder_path,j))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    k = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=os.path.join(AugFolderPath,i), save_prefix=i, save_format='jpeg'):
        k += 1
        if k > 1:
            break

In [ ]:
labels = os.listdir('/content/drive/MyDrive/machine learning/Food classification/food11/ALL_AUG_images')
len(labels)

In [ ]:
X= []
Y= []
image_size = 150
labels = os.listdir('/content/drive/MyDrive/machine learning/Food classification/food11/ALL_AUG_images')
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/machine learning/Food classification/food11/ALL_AUG_images',i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath,j))
        if img is not None:
          img = cv2.resize(img,(image_size,image_size))
          X.append(img)
          Y.append(i)




X = np.array(X)
Y = np.array(Y)

X,Y = shuffle(X,Y,random_state=0)
X.shape

In [ ]:
X=np.load('/content/drive/MyDrive/machine learning/Food classification/X.npy')
Y=np.load('/content/drive/MyDrive/machine learning/Food classification/Y.npy')

In [ ]:
from sklearn.model_selection import  train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

In [ ]:
Y_train_new = []
for i in Y_train:
    Y_train_new.append(labels.index(i))
Y_train=Y_train_new
Y_train = tf.keras.utils.to_categorical(Y_train)

Y_test_new = []
for i in Y_test:
    Y_test_new.append(labels.index(i))
Y_test=Y_test_new
Y_test = tf.keras.utils.to_categorical(Y_test)

In [ ]:
model=models.Sequential([
    #cnn
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)),
    layers.MaxPooling2D((2,2)),
    #dense
    layers.Flatten(),
    layers.Dense(100,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(80,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(50,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(40,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(25,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(11,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train) ,np.array(Y_train_new),epochs=200,batch_size=500)

In [ ]:
model.evaluate(np.array(X_test),np.array(Y_test_new))